In [ ]:
import pandas as pd 
logs = pd.read_csv("data/driving_log.csv")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns 
sns.set_style("white")
r = [1/(e+0.1) for e in logs.steering]
sns.distplot(r,kde=False)#,bins=np.linspace(-100.0,100.0,50))

In [ ]:
logs.describe()

In [ ]:
from PIL import Image

In [ ]:
Image.open("data/IMG/center_2016_12_01_13_43_21_799.jpg")

In [ ]:
im = Image.open("data/IMG/center_2016_12_01_13_43_21_799.jpg").transpose(Image.FLIP_LEFT_RIGHT)

In [ ]:
im.thumbnail((160,320), Image.ANTIALIAS)

In [ ]:
import numpy as np
print(np.shape(im))

In [ ]:
plt.imshow(im)

### Data Augmentation
1. Flip the images right to left and left to right and negate the steering angles
2. Left camera image angle + (angle delta(0.3))
3. Right camera image angle - (angle delta (0.3))
4. 


In [ ]:
X_train = []
y_train = []
for d,s in zip(logs.center,logs.steering):
    im = Image.open("./data/"+d)
    X_train.append(np.array(im))
    y_train.append(s)
    
    

In [ ]:
augmented_data = []
augmented_steer = []
for d,s in zip(logs.center,logs.steering):
    if(s != 0.0):
        im = Image.open("./data/"+d)
        #im.thumbnail((160,320), Image.ANTIALIAS)
        augmented_data.append(np.array(im.transpose(Image.FLIP_LEFT_RIGHT)))
        augmented_steer.append(s*-1.0)
    augmented_data.append(np.array(im))
    augmented_steer.append(s)
correction_factor = 0.2
for d,s in zip(logs.left,logs.steering):
    s = s +correction_factor
    
    im = Image.open("./data/"+d.strip())
    #im.thumbnail((160,320), Image.ANTIALIAS)
    augmented_data.append(np.array(im.transpose(Image.FLIP_LEFT_RIGHT)))
    augmented_steer.append(s*-1.0)
    augmented_data.append(np.array(im))
    augmented_steer.append(s)
for d,s in zip(logs.right,logs.steering):
    s = s -correction_factor
    
    im = Image.open("./data/"+d.strip())
    #im.thumbnail((160,320), Image.ANTIALIAS)
    augmented_data.append(np.array(im.transpose(Image.FLIP_LEFT_RIGHT)))
    augmented_steer.append(s*-1.0)
    augmented_data.append(np.array(im))
    augmented_steer.append(s)

In [ ]:
X_train = np.asarray(augmented_data)
y_train = np.asarray(augmented_steer)

In [ ]:
np.shape(X_train)

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten


from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Convolution2D(32,3,3,input_shape=(160,320,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64,3,3))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')

In [ ]:
model.fit(X_train,y_train,batch_size=512,validation_split=0.2,nb_epoch=5)

In [ ]:
model.save("model.h5")

In [ ]:
sns.distplot(augmented_steer,kde=False)#,bins=np.linspace(-100.0,100.0,50))